# Train prediction models for texture 
* convolutional neural network and LSTM over samples; input features_SA and features RA of size (n_pins, n_pins, n_samples) representing firing rate over time on an n_pins x n_pins square array                
* training data concatenated over speed leaving one speed out but retained for testing
* therefore total of n_speed models (here 10), one per speed
* i.e. prediction of texture independent of held-out speed

To run, first edit dir_data to path where data is stored; spaital/process_static in this directory should be run first.

Because of the large amount of data, a temporary folder is used for the training data, so it can be deleted after training the dynamic model. 

In [1]:
import os
import pickle

def open_obj(name):
    with open(name + '.pkl', 'rb') as f:
        obj = pickle.load(f)
    return obj

In [2]:
dir_data = os.environ["DATAPATH"] + r"/open/afferents-tactile-textures-jrsi2022"
dir_temp = os.environ["TEMPPATH"] + r"/dynamic"

n_frames = 10
n_speeds = 10
n_textures = 13
n_pins = 19

In [3]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session, clear_session
from keras import optimizers, regularizers, callbacks, Input, Model
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization, LSTM, TimeDistributed
from numpy.random import seed
from tensorflow import set_random_seed

C:\Users\nl13426\anaconda3\envs\tactip\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\nl13426\anaconda3\envs\tactip\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\nl13426\anaconda3\envs\tactip\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\nl13426\anaconda3\envs\tactip\lib\site-packages\tensorf

In [4]:
def define_model(X_train, y_train, X_val, y_val, es, cp):
    
    clear_session()
    
    config = tf.ConfigProto(gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)) # device_count = {'GPU': 1}
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)
    set_session(session)
    
    seed(1)
    set_random_seed(2)
    
    cnn = Sequential()
    cnn.add(Conv2D(64, (3,3), activation='relu', padding = "same", input_shape=(n_pins,n_pins,1)))
    cnn.add(MaxPooling2D((2,2)))
    cnn.add(BatchNormalization())
    cnn.add(Conv2D(128, (3,3), activation='relu', padding = "same"))
    cnn.add(BatchNormalization())
    cnn.add(Conv2D(128, (3,3), activation='relu', padding = "valid"))
    cnn.add(BatchNormalization())
    cnn.add(MaxPooling2D((2,2)))
    cnn.add(BatchNormalization())
    cnn.add(Flatten())

    rnn = Sequential()
    rnn.add(LSTM(n_frames, return_sequences = True))
    rnn.add(LSTM(n_frames, return_sequences = True))
    rnn.add(LSTM(n_frames))

    dense = Sequential()
    dense.add(Dropout(0.4))
    dense.add(Dense(32, kernel_regularizer=regularizers.l2(0.005), activation='relu'))
    dense.add(BatchNormalization())
    dense.add(Dropout(0.2))
    dense.add(Dense(16, kernel_regularizer=regularizers.l2(0.005), activation='relu'))
    dense.add(BatchNormalization())
    dense.add(Dropout(0.2))
    dense.add(Dense(n_textures, kernel_regularizer=regularizers.l2(0.005), activation='softmax'))

    main_input = Input(shape=(n_frames, n_pins, n_pins, 1))

    model = TimeDistributed(cnn)(main_input) 
    model = rnn(model)
    model = dense(model) 

    final_model = Model(inputs=main_input, outputs=model)
    final_model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=1e-4), metrics=['accuracy'])
    final_model.fit(X_train, y_train, validation_data = (X_val,y_val), epochs=150, batch_size=64, shuffle=True, callbacks=[es,cp])

# leave-one-speed-out

## SA

In [5]:
for i in range(n_speeds):
    data_set = dir_temp + rf"/{i}"
    X_train = open_obj(data_set + r"/X_train_SA")
    X_val = open_obj(data_set + r"/X_val_SA")
    y_train = open_obj(data_set + r"/y_train_SA")
    y_val = open_obj(data_set + r"/y_val_SA")
    
    dir_model = dir_data + rf"/models/dynamic_SA/{i}"
    os.makedirs(dir_model, exist_ok=True)
    cp = callbacks.ModelCheckpoint(dir_model + r"/model_{epoch:02d}_{val_accuracy:.2f}.hdf5", monitor='val_accuracy', save_best_only=True)
    es = callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)
    define_model(X_train, y_train[1], X_val, y_val[1], es, cp)



Train on 58500 samples, validate on 3250 samples
Epoch 1/150
58500/58500 [==============================] - 62s 1ms/step - loss: 2.4744 - accuracy: 0.2589 - val_loss: 2.4514 - val_accuracy: 0.2711
Epoch 2/150
58500/58500 [==============================] - 55s 934us/step - loss: 1.6706 - accuracy: 0.5358 - val_loss: 2.4614 - val_accuracy: 0.2978
Epoch 3/150
58500/58500 [==============================] - 55s 932us/step - loss: 1.3046 - accuracy: 0.6831 - val_loss: 2.5428 - val_accuracy: 0.3271
Epoch 4/150
58500/58500 [==============================] - 55s 935us/step - loss: 1.0743 - accuracy: 0.7706 - val_loss: 2.6159 - val_accuracy: 0.3172
Epoch 5/150
58500/58500 [==============================] - 54s 931us/step - loss: 0.9058 - accuracy: 0.8239 - val_loss: 2.6536 - val_accuracy: 0.3394
Epoch 6/150
58500/58500 [==============================] - 55s 935us/step - loss: 0.7934 - accuracy: 0.8516 - val_loss: 2.6652 - val_accuracy: 0.3646
Epoch 7/150
58500/58500 [==========================

## RA

In [7]:
for i in range(n_speeds):
    data_set = dir_temp + rf"/{i}"
    X_train = open_obj(data_set + r"/X_train_RA")
    X_val = open_obj(data_set + r"/X_val_RA")
    y_train = open_obj(data_set + r"/y_train_RA")
    y_val = open_obj(data_set + r"/y_val_RA")
    
    dir_model = dir_data + rf"/models/dynamic_RA/{i}"
    os.makedirs(dir_model)
    cp = callbacks.ModelCheckpoint(dir_model + r"/model_{epoch:02d}_{val_accuracy:.2f}.hdf5", monitor='val_accuracy', save_best_only=True)
    es = callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)
    define_model(X_train, y_train[1], X_val, y_val[1], es, cp)

Train on 58500 samples, validate on 3250 samples
Epoch 1/150
58500/58500 [==============================] - 63s 1ms/step - loss: 2.5954 - accuracy: 0.2158 - val_loss: 2.5357 - val_accuracy: 0.2203
Epoch 2/150
58500/58500 [==============================] - 62s 1ms/step - loss: 2.1575 - accuracy: 0.3455 - val_loss: 2.5127 - val_accuracy: 0.2357
Epoch 3/150
58500/58500 [==============================] - 62s 1ms/step - loss: 1.9174 - accuracy: 0.4255 - val_loss: 2.5300 - val_accuracy: 0.2569
Epoch 4/150
58500/58500 [==============================] - 61s 1ms/step - loss: 1.7281 - accuracy: 0.4945 - val_loss: 2.4946 - val_accuracy: 0.2732
Epoch 5/150
58500/58500 [==============================] - 61s 1ms/step - loss: 1.5690 - accuracy: 0.5474 - val_loss: 2.4538 - val_accuracy: 0.2898
Epoch 6/150
58500/58500 [==============================] - 62s 1ms/step - loss: 1.4341 - accuracy: 0.5904 - val_loss: 2.5118 - val_accuracy: 0.2926
Epoch 7/150
58500/58500 [==============================] - 61s 